##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 時系列予測

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/structured_data/time_series"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/time_series.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/structured_data/time_series.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/structured_data/time_series.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Note: これらのドキュメントは私たちTensorFlowコミュニティが翻訳したものです。コミュニティによる 翻訳は**ベストエフォート**であるため、この翻訳が正確であることや[英語の公式ドキュメント](https://www.tensorflow.org/?hl=en)の 最新の状態を反映したものであることを保証することはできません。 この翻訳の品質を向上させるためのご意見をお持ちの方は、GitHubリポジトリ[tensorflow/docs](https://github.com/tensorflow/docs)にプルリクエストをお送りください。 コミュニティによる翻訳やレビューに参加していただける方は、 [docs-ja@tensorflow.org メーリングリスト](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ja)にご連絡ください。

このチュートリアルは、回帰型ニューラルネットワーク(Recurrent Neural Networks) (RNNs)を利用した時系列予測の紹介です。二つのパートに分かれています：最初は、一つの時系列の変数の予測を行い、次に複数の時系列変数の予測を行います。

In [ ]:
import tensorflow as tf

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## 気象データセット
このチュートリアルでは、<a href="https://www.bgc-jena.mpg.de" class="external">マックスプランク生物地球科学研究所</a>が記録した<a href="https://www.bgc-jena.mpg.de/wetter/" class="external">[気象時系列データセット</a>を利用します。

このデータセットは、気温、気圧、湿度などの14種の異なる特性を含んでいます。2003年から10分おきのデータセットが収集されています。効率を上げるために、2009年から2016年の間に収集されたデータのみを使用します。このセクションでは、François Cholletの著書[Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python)で用意されたデータセットを使います。

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)

In [ ]:
df = pd.read_csv(csv_path)

ざっとデータを見てみましょう。

In [ ]:
df.head()

上記のように、観測は10分ごとに記録されます。 これは、1時間の間に6つの観測があることを意味します。 同様に、1日には144（6x24）の観測が含まれます。

特定の時間を想定して、6時間後の気温を予測するとします。この予測を行うには、5日間の観測を使用することを選択します。したがって、モデルをトレーニングするために、最後の720（5x144）観測を含むウィンドウを作成します。このような多くの構成が可能であり、このデータセットを実験するのに適しています。

以下の関数は、モデルが学習するための上記の時間ウィンドウを返します。パラメータ `history_size`は、過去の情報ウィンドウのサイズです。`target_size`は予測する必要があるラベルです。

In [ ]:
def univariate_data(dataset, start_index, end_index, history_size, target_size):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i)
    # Reshape data from (history_size,) to (history_size, 1)
    data.append(np.reshape(dataset[indices], (history_size, 1)))
    labels.append(dataset[i+target_size])
  return np.array(data), np.array(labels)

次の両方のチュートリアルでは、データの最初の300,000行がトレーニングデータセットになり、残りは検証データセットになります。これは、約2100日分のトレーニングデータに相当します。

In [ ]:
TRAIN_SPLIT = 300000

再現性を確保するためのシードの設定。

In [ ]:
tf.random.set_seed(13)

## パート1： 一変量時系列を予測する
最初に、単一の特徴（温度）のみを使用してモデルをトレーニングし、それを使用して将来のその値の予測を行います。

まず、データセットから温度のみを抽出します。

In [ ]:
uni_data = df['T (degC)']
uni_data.index = df['Date Time']
uni_data.head()

このデータが経時的にどのように見えるかを見てみましょう。

In [ ]:
uni_data.plot(subplots=True)

In [ ]:
uni_data = uni_data.values

ニューラルネットワークをトレーニングする前に、特徴をスケーリングすることが重要です。標準化は、平均を減算して各特徴の標準偏差で除算することにより、このスケーリングを行う一般的な方法です。値を[0,1]の範囲に再スケーリングする `tf.keras.utils.normalize`メソッドを使用することもできます。

Note: 平均と標準偏差は、トレーニングデータを使用してのみ計算する必要があります。

In [ ]:
uni_train_mean = uni_data[:TRAIN_SPLIT].mean()
uni_train_std = uni_data[:TRAIN_SPLIT].std()

データを標準化しましょう。

In [ ]:
uni_data = (uni_data-uni_train_mean)/uni_train_std

単変量モデルのデータを作成しましょう。 パート1では、モデルには最後に記録された20個の温度観測が与えられ、次のタイムステップで温度を予測することを学習する必要があります。

In [ ]:
univariate_past_history = 20
univariate_future_target = 0

x_train_uni, y_train_uni = univariate_data(uni_data, 0, TRAIN_SPLIT,
                                           univariate_past_history,
                                           univariate_future_target)
x_val_uni, y_val_uni = univariate_data(uni_data, TRAIN_SPLIT, None,
                                       univariate_past_history,
                                       univariate_future_target)

これは、 `univariate_data`関数が返すものです。

In [ ]:
print ('Single window of past history')
print (x_train_uni[0])
print ('\n Target temperature to predict')
print (y_train_uni[0])

データが作成されたので、1つの例を見てみましょう。 ネットワークに提供される情報は青で表示され、赤い十字の値を予測する必要があります。

In [ ]:
def create_time_steps(length):
  return list(range(-length, 0))

In [ ]:
def show_plot(plot_data, delta, title):
  labels = ['History', 'True Future', 'Model Prediction']
  marker = ['.-', 'rx', 'go']
  time_steps = create_time_steps(plot_data[0].shape[0])
  if delta:
    future = delta
  else:
    future = 0

  plt.title(title)
  for i, x in enumerate(plot_data):
    if i:
      plt.plot(future, plot_data[i], marker[i], markersize=10,
               label=labels[i])
    else:
      plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])
  plt.legend()
  plt.xlim([time_steps[0], (future+5)*2])
  plt.xlabel('Time-Step')
  return plt

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0]], 0, 'Sample Example')

### ベースライン
モデルのトレーニングに進む前に、まず単純なベースラインを設定しましょう。 入力ポイントが与えられると、ベースラインメソッドはすべての履歴を調べ、次のポイントを最後の20観測の平均であると予測します。

In [ ]:
def baseline(history):
  return np.mean(history)

In [ ]:
show_plot([x_train_uni[0], y_train_uni[0], baseline(x_train_uni[0])], 0,
           'Baseline Prediction Example')

リカレントニューラルネットワークを使用して、このベースラインを打つことができるかどうか見てみましょう。

### リカレントニューラルネットワーク
再帰型ニューラルネットワーク（RNN）は、時系列データに適したタイプのニューラルネットワークです。 RNNは時系列を段階的に処理し、これまでに確認した情報を要約した内部状態を維持します。詳細は [RNN チュートリアル](https://www.tensorflow.org/tutorials/sequences/recurrent)を読んでください。このチュートリアルでは、Long Short Term Memory ([LSTM](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/LSTM)) と呼ばれる特殊なRNNレイヤーを使用します。

ここで、 `tf.data`を使用してデータセットをシャッフル、バッチ処理、キャッシュします。

In [ ]:
BATCH_SIZE = 256
BUFFER_SIZE = 10000

train_univariate = tf.data.Dataset.from_tensor_slices((x_train_uni, y_train_uni))
train_univariate = train_univariate.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_univariate = tf.data.Dataset.from_tensor_slices((x_val_uni, y_val_uni))
val_univariate = val_univariate.batch(BATCH_SIZE).repeat()

次の視覚化は、バッチ処理後のデータの表現方法を理解するのに役立ちます。

![時系列](images/time_series.png)

LSTMは、与えられているデータの入力形状を必要とすることがわかります。

In [ ]:
simple_lstm_model = tf.keras.models.Sequential([
    tf.keras.layers.LSTM(8, input_shape=x_train_uni.shape[-2:]),
    tf.keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mae')

サンプル予測を作成して、モデルの出力を確認してみましょう。

In [ ]:
for x, y in val_univariate.take(1):
    print(simple_lstm_model.predict(x).shape)

ここでモデルをトレーニングしましょう。 データセットのサイズが大きいため、時間を節約するために、通常行われる完全なトレーニングデータではなく、各エポックは200ステップだけ実行されます。

In [ ]:
EVALUATION_INTERVAL = 200
EPOCHS = 10

simple_lstm_model.fit(train_univariate, epochs=EPOCHS,
                      steps_per_epoch=EVALUATION_INTERVAL,
                      validation_data=val_univariate, validation_steps=50)

#### 単純なLSTMモデルを使用して予測する
簡単なLSTMをトレーニングしたので、いくつかの予測をしてみましょう。

In [ ]:
for x, y in val_univariate.take(3):
  plot = show_plot([x[0].numpy(), y[0].numpy(),
                    simple_lstm_model.predict(x)[0]], 0, 'Simple LSTM model')
  plot.show()

これはベースラインよりも良く見えます。 基本を理解したところで、パート2に進みましょう。ここでは、多変量時系列を操作します。

## Part 2: Forecast a multivariate time series

## パート2： 多変量時系列を予測する

The original dataset contains fourteen features. For simplicity, this section considers only three of the original fourteen. The features used are air temperature, atmospheric pressure, and air density. 

To use more features, add their names to this list.

元のデータセットには14の特徴が含まれています。 簡単にするために、このセクションでは元の14のうち3つだけを考慮します。 使用される機能は、気温、大気圧、および空気密度です。

さらに多くの機能を使用するには、その名前をこのリストに追加します。

In [ ]:
features_considered = ['p (mbar)', 'T (degC)', 'rho (g/m**3)']

In [ ]:
features = df[features_considered]
features.index = df['Date Time']
features.head()

Let's have a look at how each of these features vary across time.

これらの各機能が時間の経過とともにどのように変化するかを見てみましょう。

In [ ]:
features.plot(subplots=True)

As mentioned, the first step will be to standardize the dataset using the mean and standard deviation of the training data.

前述のように、最初のステップは、トレーニングデータの平均と標準偏差を使用してデータセットを標準化することです。

In [ ]:
dataset = features.values
data_mean = dataset[:TRAIN_SPLIT].mean(axis=0)
data_std = dataset[:TRAIN_SPLIT].std(axis=0)

In [ ]:
dataset = (dataset-data_mean)/data_std

### Single step model
In a single step setup, the model learns to predict a single point in the future based on some history provided.

The below function performs the same windowing task as above, however, here it samples the past observation based on the step size given.

### シングルステップモデル
シングルステップセットアップでは、モデルは、提供された履歴に基づいて将来の単一のポイントを予測することを学習します。

以下の関数は、上記と同じウィンドウ処理タスクを実行しますが、ここでは、指定されたステップサイズに基づいて過去の観測値をサンプリングします。

In [1]:
def multivariate_data(dataset, target, start_index, end_index, history_size,
                      target_size, step, single_step=False):
  data = []
  labels = []

  start_index = start_index + history_size
  if end_index is None:
    end_index = len(dataset) - target_size

  for i in range(start_index, end_index):
    indices = range(i-history_size, i, step)
    data.append(dataset[indices])

    if single_step:
      labels.append(target[i+target_size])
    else:
      labels.append(target[i:i+target_size])

  return np.array(data), np.array(labels)

In this tutorial, the network is shown data from the last five (5) days, i.e. 720 observations that are sampled every hour. The sampling is done every one hour since a drastic change is not expected within 60 minutes. Thus, 120 observation represent history of the last five days.  For the single step prediction model, the label for a datapoint is the temperature 12 hours into the future. In order to create a label for this, the temperature after 72(12*6) observations is used.

このチュートリアルでは、ネットワークには過去5日間のデータが表示されます。つまり、1時間ごとにサンプリングされる720観測です。 60分以内に大幅な変化が予想されないため、サンプリングは1時間ごとに行われます。 したがって、120の観測は過去5日間の履歴を表します。 シングルステップ予測モデルの場合、データポイントのラベルは12時間後の気温です。 このためのラベルを作成するために、72（12 * 6）観測後の温度が使用されます。

In [ ]:
past_history = 720
future_target = 72
STEP = 6

x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0,
                                                   TRAIN_SPLIT, past_history,
                                                   future_target, STEP,
                                                   single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1],
                                               TRAIN_SPLIT, None, past_history,
                                               future_target, STEP,
                                               single_step=True)

Let's look at a single data-point.


単一のデータポイントを見てみましょう。


In [ ]:
print ('Single window of past history : {}'.format(x_train_single[0].shape))

In [ ]:
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

In [ ]:
single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32,
                                           input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))

single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')

Let's check out a sample prediction.

サンプル予測をチェックしてみましょう。

In [ ]:
for x, y in val_data_single.take(1):
  print(single_step_model.predict(x).shape)

In [ ]:
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch=EVALUATION_INTERVAL,
                                            validation_data=val_data_single,
                                            validation_steps=50)

In [ ]:
def plot_train_history(history, title):
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(loss))

  plt.figure()

  plt.plot(epochs, loss, 'b', label='Training loss')
  plt.plot(epochs, val_loss, 'r', label='Validation loss')
  plt.title(title)
  plt.legend()

  plt.show()

In [ ]:
plot_train_history(single_step_history,
                   'Single Step Training and validation loss')

#### Predict a single step future
Now that the model is trained, let's make a few sample predictions. The model is given the history of three features over the past five days sampled every hour (120 data-points), since the goal is to predict the temperature, the plot only displays the past temperature. The prediction is made one day into the future (hence the gap between the history and prediction). 

#### シングルステップの先を予測する
モデルがトレーニングされたので、サンプル予測をいくつか行ってみましょう。 モデルには、毎時サンプリングされた過去5日間の3つの特徴の履歴が与えられます（120データポイント）。目標は温度を予測することであるため、プロットは過去の温度のみを表示します。 予測は1日後に行われます（したがって、履歴と予測の間のギャップがあります）。

In [ ]:
for x, y in val_data_single.take(3):
  plot = show_plot([x[0][:, 1].numpy(), y[0].numpy(),
                    single_step_model.predict(x)[0]], 12,
                   'Single Step Prediction')
  plot.show()

### Multi-Step model
In a multi-step prediction model, given a past history, the model needs to learn to predict a range of future values. Thus, unlike a single step model, where only a single future point is predicted, a multi-step model predict a sequence of the future.

For the multi-step model, the training data again consists of recordings over the past five days sampled every hour. However, here, the model needs to learn to predict the temperature for the next 12 hours. Since an obversation is taken every 10 minutes, the output is 72 predictions. For this task, the dataset needs to be prepared accordingly, thus the first step is just to create it again, but with a different target window.

### マルチステップモデル
マルチステップ予測モデルでは、過去の履歴を前提として、モデルは将来の値の範囲を予測することを学習する必要があります。 したがって、単一の未来点のみが予測されるシングルステップモデルとは異なり、マルチステップモデルは未来のシーケンスを予測します。

マルチステップモデルの場合、トレーニングデータは、毎時間サンプリングされた過去5日間の記録で構成されます。 ただし、ここでは、モデルは次の12時間の温度を予測することを学習する必要があります。 観測(訳者注；obversationはobservationの誤り)は10分ごとに行われるため、出力は72予測です。 このタスクでは、それに応じてデータセットを準備する必要があります。したがって、最初のステップは、データセットを再度作成することですが、ターゲットウィンドウは異なります。

In [ ]:
future_target = 72
x_train_multi, y_train_multi = multivariate_data(dataset, dataset[:, 1], 0,
                                                 TRAIN_SPLIT, past_history,
                                                 future_target, STEP)
x_val_multi, y_val_multi = multivariate_data(dataset, dataset[:, 1],
                                             TRAIN_SPLIT, None, past_history,
                                             future_target, STEP)

Let's check out a sample data-point.

サンプルのデータポイントを確認してみましょう。

In [ ]:
print ('Single window of past history : {}'.format(x_train_multi[0].shape))
print ('\n Target temperature to predict : {}'.format(y_train_multi[0].shape))

In [ ]:
train_data_multi = tf.data.Dataset.from_tensor_slices((x_train_multi, y_train_multi))
train_data_multi = train_data_multi.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_multi = tf.data.Dataset.from_tensor_slices((x_val_multi, y_val_multi))
val_data_multi = val_data_multi.batch(BATCH_SIZE).repeat()

Plotting a sample data-point.

サンプルデータポイントのプロット。

In [ ]:
def multi_step_plot(history, true_future, prediction):
  plt.figure(figsize=(12, 6))
  num_in = create_time_steps(len(history))
  num_out = len(true_future)

  plt.plot(num_in, np.array(history[:, 1]), label='History')
  plt.plot(np.arange(num_out)/STEP, np.array(true_future), 'bo',
           label='True Future')
  if prediction.any():
    plt.plot(np.arange(num_out)/STEP, np.array(prediction), 'ro',
             label='Predicted Future')
  plt.legend(loc='upper left')
  plt.show()

In this plot and subsequent similar plots, the history and the future data are sampled every hour.

このプロットとその後の同様のプロットでは、履歴と将来のデータが1時間ごとにサンプリングされます。

In [ ]:
for x, y in train_data_multi.take(1):
  multi_step_plot(x[0], y[0], np.array([0]))

Since the task here is a bit more complicated than the previous task, the model now consists of two LSTM layers. Finally, since 72 predictions are made, the dense layer outputs 72 predictions.

ここでのタスクは前のタスクよりも少し複雑であるため、モデルは2つのLSTMレイヤーで構成されています。 最後に、72個の予測が行われるため、denseレイヤーは72個の予測を出力します。

In [ ]:
multi_step_model = tf.keras.models.Sequential()
multi_step_model.add(tf.keras.layers.LSTM(32,
                                          return_sequences=True,
                                          input_shape=x_train_multi.shape[-2:]))
multi_step_model.add(tf.keras.layers.LSTM(16, activation='relu'))
multi_step_model.add(tf.keras.layers.Dense(72))

multi_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss='mae')

Let's see how the model predicts before it trains.

トレーニングする前にモデルがどのように予測するかを見てみましょう。

In [ ]:
for x, y in val_data_multi.take(1):
  print (multi_step_model.predict(x).shape)

In [ ]:
multi_step_history = multi_step_model.fit(train_data_multi, epochs=EPOCHS,
                                          steps_per_epoch=EVALUATION_INTERVAL,
                                          validation_data=val_data_multi,
                                          validation_steps=50)

In [ ]:
plot_train_history(multi_step_history, 'Multi-Step Training and validation loss')

#### Predict a multi-step future
Let's now have a look at how well your network has learnt to predict the future.

#### マルチステップの未来を予測する
次に、ネットワークが将来を予測するためにどれだけよく学んだかを見てみましょう。

In [ ]:
for x, y in val_data_multi.take(3):
  multi_step_plot(x[0], y[0], multi_step_model.predict(x)[0])

## Next steps
This tutorial was a quick introduction to time series forecasting using an RNN. You may now try to predict the stock market and become a billionaire.

In addition, you may also write a generator to yield data (instead of the uni/multivariate_data function), which would be more memory efficient. You may also check out this [time series windowing](https://www.tensorflow.org/guide/data#time_series_windowing) guide and use it in this tutorial.

For further understanding, you may read Chapter 15 of [Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/), 2nd Edition and Chapter 6 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python).

## 次のステップ
このチュートリアルは、RNNを使用した時系列予測の簡単な紹介です。 あなたは今、株式市場を予測して億万長者になることを試みるかもしれません。

さらに、(uni/multivariate_data関数の代わりに)データを生成するジェネレーターを作成することもできます。これにより、メモリ効率が向上します。この[時系列のウィンドウ処理](https://www.tensorflow.org/guide/data#time_series_windowing)ガイドを確認して、このチュートリアルで使用することもできます。

詳細については、第15章[Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)、と[Pythonによる深層学習](https://www.manning.com/books/deep-learning-with-python)第2版の第6章を参照してください。